In [1]:
import pandas as pd
import json
from datetime import datetime

# Step 1: Sort Data by Timestamp
tweet_raw_file_dir = "C:\\Users\\jesse\\Desktop\\DDHW\\Testing\\20231021_merged_data.csv"
df = pd.read_csv(tweet_raw_file_dir)
df['Timestamp'] = pd.to_datetime(df['Timestamp'])
df = df.sort_values(by='Timestamp')
df.head(len(df))

,Timestamp,Username,Tweet Content,Retweets,Likes,Replies,scores
5,2023-10-10 12:00:10,BTC_Archive,RT @Volcano_Energy: Are you ready for our firs...,32,0,0,0.112504
87,2023-10-10 13:14:25,DocumentingBTC,"Fidelity manages $4,500,000,000,000.\n\n43,000...",1507,4619,157,0.596841
89,2023-10-10 13:14:26,DocumentingBTC,2/ Fidelity finds #bitcoin is the best money.\...,43,239,19,0.942388
88,2023-10-10 13:14:26,DocumentingBTC,"3/ Fidelity explains ""Virtuous Cycle of Bitcoi...",27,202,7,0.427571
91,2023-10-10 13:14:27,DocumentingBTC,4/ Fidelity compares #bitcoin to the wheel.\n\...,33,226,4,-0.034173
...,...,...,...,...,...,...,...
212,2023-10-17 11:11:13,bitcoinlfgo,RT @bitcoinlfgo: BREAKING 🚨 $BTC \n\n#BITCOIN ...,145,0,0,0.425457
60,2023-10-17 11:17:53,BTC_Archive,"RT @BTC_Archive: When the banks crash, #Bitcoi...",334,0,0,0.736291
61,2023-10-17 11:21:50,BTC_Archive,"When BlackRock CEO says ‘crypto’, he really me...",46,310,60,0.158183
108,2023-10-17 11:25:19,DocumentingBTC,RT @DocumentingBTC: ₿𝗥𝗘𝗔𝗞𝗜𝗡𝗚: New record #bitc...,512,0,0,0.497934


In [2]:
# Step 2: Calculate Sentiment Index
df['Sentiment_Index'] = df['Likes'] * df['scores']

# Step 3: Group Data by Date and Sum Sentiment_Index
# Extract the date from the 'Timestamp' column
df['Date'] = df['Timestamp'].dt.date

# Group the data by date and sum the 'Sentiment_Index' for each date
processed_data = df.groupby('Date')['Sentiment_Index'].sum().reset_index()

# Step 4: Create and Save Processed Data with a Specified Filename
target_directory = "C:\\Users\\jesse\\Desktop\\DDHW\\Testing\\"
output_file_path = target_directory + 'processed_sentiment_index_by_date.csv'

# Save the processed data to a new CSV file without index column
processed_data.to_csv(output_file_path, index=False)

df = pd.read_csv(output_file_path)
df.head(len(df))

,Date,Sentiment_Index
0,2023-10-10,11545.618104
1,2023-10-11,5105.910992
2,2023-10-12,4321.347785
3,2023-10-13,12474.556048
4,2023-10-14,3330.625824
5,2023-10-15,8110.174505
6,2023-10-16,11217.516881
7,2023-10-17,4431.445887


In [3]:
# Convert JSON data to CSV

# Read JSON data from a file
bitcoin_price_file_dir = "C:\\Users\\jesse\\Desktop\\DDHW\\Testing\\price.json"
with open(bitcoin_price_file_dir, 'r') as file:
    json_data = file.read()

# Parse JSON data
data = json.loads(json_data)

# Extract 'Date' and 'Price' data
price_data = data.get('market-price', [])
date_price_data = [{'Date': datetime.fromtimestamp(item['x'] / 1000).strftime('%Y-%m-%d'), 'Price': item['y']} for item in price_data]

# Create a DataFrame
df = pd.DataFrame(date_price_data)

# Define the directory where you want to save the CSV file
output_directory = "C:\\Users\\jesse\\Desktop\\DDHW\\Testing\\"

# Construct the full file path for the CSV file, using the JSON file's name
json_filename = bitcoin_price_file_dir.split("\\")[-1]  # Get the JSON file name
csv_filename = output_directory + json_filename.replace(".json", ".csv")  # Replace the extension

# Save DataFrame as a CSV file
df.to_csv(csv_filename, index=False)

In [4]:
bitcoin_price_file_dir = "C:\\Users\\jesse\\Desktop\\DDHW\\Testing\\price.csv"
df = pd.read_csv(bitcoin_price_file_dir)
df.head(len(df))

,Date,Price
0,2023-09-22,26569.60
1,2023-09-23,26581.86
2,2023-09-24,26580.72
3,2023-09-25,26252.75
4,2023-09-26,26293.29
5,2023-09-27,26212.82
6,2023-09-28,26357.65
7,2023-09-29,27021.15
8,2023-09-30,26916.91
9,2023-10-01,26970.43


In [5]:
# Step 5: Merge Sentiment_Index with Price_Next_Day

# Read data from both files
sentiment_df = pd.read_csv(output_file_path)
bitcoin_df = pd.read_csv(bitcoin_price_file_dir)

# Convert date columns to datetime format for consistency
sentiment_df['Date'] = pd.to_datetime(sentiment_df['Date'])
bitcoin_df['Date'] = pd.to_datetime(bitcoin_df['Date'])

# Shift the Bitcoin price values by one day to create the "price_next_day" column
bitcoin_df['Price'] = bitcoin_df['Price'].shift(-1)  # Shift prices by one day

# Merge data based on the 'Date' column
merged_df = sentiment_df.merge(bitcoin_df, on='Date', how='left')

# Overwrite the existing file with the merged data
merged_df.to_csv(output_file_path, index=False)

df = df = pd.read_csv(output_file_path)
df.head(len(df))

,Date,Sentiment_Index,Price
0,2023-10-10,11545.618104,27395.65
1,2023-10-11,5105.910992,26867.79
2,2023-10-12,4321.347785,26758.66
3,2023-10-13,12474.556048,26866.51
4,2023-10-14,3330.625824,26855.89
5,2023-10-15,8110.174505,27167.87
6,2023-10-16,11217.516881,28517.32
7,2023-10-17,4431.445887,28413.17
